In [3]:
!pip install chromadb langchain tiktoken transformers
!pip install sentence-transformers
!pip install -U langchain-community
!pip install langchain-groq

  Using cached langchain_core-0.3.79-py3-none-any.whl.metadata (3.2 kB)
  Using cached langchain_text_splitters-0.3.11-py3-none-any.whl.metadata (1.8 kB)
Using cached langchain_core-0.3.79-py3-none-any.whl (449 kB)
Using cached langchain_text_splitters-0.3.11-py3-none-any.whl (33 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.6
    Uninstalling langchain-core-1.0.6:
      Successfully uninstalled langchain-core-1.0.6
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 1.0.0
    Uninstalling langchain-text-splitters-1.0.0:
      Successfully uninstalled langchain-text-splitters-1.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-c

In [4]:
# !wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip

In [5]:
# !unzip -q /kaggle/working/new_articles.zip -d new_articles

In [6]:
import langchain
import chromadb
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader,PyPDFLoader

In [7]:
loader = DirectoryLoader("/content/sample_data", glob = "**/*.pdf", loader_cls=PyPDFLoader)

In [9]:
!pip install pypdf
document = loader.load()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 6.7 MB/s eta 0:00:00


In [10]:
import langchain
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 3000, chunk_overlap = 200)
text = text_splitter.split_documents(document)

In [15]:
text[4]

Document(metadata={'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2022-05-17T18:01:22-07:00', 'author': '', 'title': '', 'subject': '', 'keywords': '', 'moddate': '2022-05-17T18:01:22-07:00', 'trapped': '/False', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'source': '/content/sample_data/main_notes_ML.pdf', 'total_pages': 216, 'page': 4, 'page_label': '4'}, page_content='CS229 Spring 2022 4\nV Reinforcement Learning and Control 175\n15 Reinforcement learning 176\n15.1 Markov decision processes . . . . . . . . . . . . . . . . . . . . 177\n15.2 Value iteration and policy iteration . . . . . . . . . . . . . . . 179\n15.3 Learning a model for an MDP . . . . . . . . . . . . . . . . . . 181\n15.4 Continuous state MDPs . . . . . . . . . . . . . . . . . . . . . 183\n15.4.1 Discretization . . . . . . . . . . . . . . . . . . . . . . . 183\n15.4.2 Value function approximation . . . . . . . . . . . . 

In [16]:
len(text)

217

In [18]:
!pip install langchain_huggingface
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
persist_directory = 'db'
embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
vectordb = Chroma.from_documents(documents=text[:5],
                                 embedding=embedding,
                                 persist_directory=persist_directory)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

/tmp/ipython-input-2004002305.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory,


In [20]:
retriever = vectordb.as_retriever()

In [22]:
docs = retriever.invoke("What is LMS algorithm?")
docs

[Document(metadata={'creator': 'LaTeX with hyperref', 'page_label': '1', 'keywords': '', 'moddate': '2022-05-17T18:01:22-07:00', 'author': '', 'producer': 'pdfTeX-1.40.21', 'source': '/content/sample_data/main_notes_ML.pdf', 'subject': '', 'trapped': '/False', 'total_pages': 216, 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'title': '', 'creationdate': '2022-05-17T18:01:22-07:00', 'page': 1}, page_content='Contents\nI Supervised learning 5\n1 Linear regression 8\n1.1 LMS algorithm . . . . . . . . . . . . . . . . . . . . . . . . . . 9\n1.2 The normal equations . . . . . . . . . . . . . . . . . . . . . . . 13\n1.2.1 Matrix derivatives . . . . . . . . . . . . . . . . . . . . . 13\n1.2.2 Least squares revisited . . . . . . . . . . . . . . . . . . 14\n1.3 Probabilistic interpretation . . . . . . . . . . . . . . . . . . . . 15\n1.4 Locally weighted linear regression (optional reading) . . . . . . 17\n2 Classiﬁcation and logistic 

In [23]:
retriever.search_type

'similarity'

In [32]:
!pip show langchain

Name: langchain
Version: 0.3.27
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: 


In [34]:
!pip install langchain-schema langchain-core
# from langchain.schema import BaseMemory
# from langchain.chains import RetrievalQA

ERROR: Could not find a version that satisfies the requirement langchain-schema (from versions: none)
ERROR: No matching distribution found for langchain-schema


In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0,
    groq_api_key='',
    model_name="llama-3.1-70b-versatile"
)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
query = "How much money did Microsoft invested?"
llm_response = qa_chain(query)
llm_response['result']

"According to the provided context, Microsoft's investment is believed to be around $10 billion."